__Obiettivo__

Estrarre alcuni metadati relativi ai file PDF contenuti nella cartella _articlesGrobid_, mediante l'impiego di apposite librerie.

Nuovamente, i dati estratti si suddividono in:
- __DOI__, Digital Object Identifier, identificativo univoco di risorse digitali
- __Title__, titolo dell'articolo scientifico
- __Authors__, autore/autori partecipanti alla stesura del paper preso in considerazione
- __Abstract__, piccolo riassunto del documento, privo di interpretazioni o critiche

Il notebook ricalca lo stesso obiettivo disposto dal file __metadata.ipynb__, ma prendendo in esame un bacino molto più elevato rispetto ai dieci file contenuti in _articles_.
Proprio per questa principale ragione, è stata adeguata una specifica libreria denominata __Grobid__, acronimo di _Generation Of Bibliographic Data_.

_Grobid_ è una libreria di _machine learning_, il cui scopo consiste nell'estrazione e conversione di un file PDF in un _formato struttutato XML_. Le funzionalità sviluppate permettono di acquisire un insieme di dati già confezionato, disposti secondo l'ordine gerarchico espresso dal _markup language_ in questione. 

Infatti, come presentato negli snippet di codice successivi, è stata adeguata la libreria _BeatifulSoupe_ affinchè il risultato ottenuto da _Grobid_, memorizzato all'interno di un'apposita _directory_, fosse convertito in un formato idoneo al linguaggio di programmazione utilizzato.

In [16]:
pdf_path = "../papers/articles/"

input_dir = "../articlesGrobid/"
output_dir = "../TEI/"

Le due classi presentate sono utilizzate rispettivamente per:
- __Author__, classe rappresentativa di tutti gli _autori_ degli articoli scientifici
- __Metadata__, ciascun oggetto istanziato della classe rappresenta i _metadati_ ottenuti di ogni singolo file posto all'interno della cartella _articlesGrobid_

In [17]:
class Author:
    def __init__(self, forename: str, surname: str):
        self.forename = forename
        self.surname = surname

    def to_unique(self) -> str | None:
        return self.forename + " " + self.surname or None

In [18]:
from typing import List, Dict

class Metadata:
    def __init__(self, DOI: str, path: str, title: str, author: List[Author], abstract: str, introduction: str):
        self.DOI = DOI
        self.path = path
        self.title = title
        self.author = author
        self.abstract = abstract
        self.introduction = introduction

    def get_dict(self) -> Dict[str, Dict[str, str | List[Author] | None]]:
        return {
            self.path: {
                "DOI": self.DOI,
                "Title": self.title,
                "Author": [item.to_unique() for item in self.author],
                "Abstract": self.abstract,
                "Introduction": self.introduction
            }
        }

In [19]:
import os
import shutil

# Took a certain range of PDF from the papers to articlesGrobid
def define_grobid_dir(main_path: str):
    for path in os.listdir(main_path):
        try:
            year = int(path[:2])

            if year == 0 and path.endswith(".pdf"):
                shutil.copy(main_path + path, input_dir)
        except ValueError as e:
            continue

if not os.path.exists(input_dir):
    os.makedirs(input_dir)
    define_grobid_dir(pdf_path)

if not os.path.exists(output_dir):
    os.makedirs(output_dir)

Prima di eseguire lo snippet di codice seguente, è necessario abilitare la _Web API_ di _Grobid_. Ciò può avvenire in due modi, suddivisi in:  
- Installazione di _Grobid_ localmente, in cui è richiesta la previa presenza del _JDK (Java Development Kit)_ e di _Gradle_
- Utilizzo di _Docker_ tramite l'_image_ fornita dalla documentazione, necessaria per realizzare il _container_ associato, ossia l'istanza eseguibile della stessa immagine

La scelta progettuale è ricaduta su _Docker_, data l'estrema semplicità garantita dal tool. Di seguito, è riportato il comando necessario per abilitare la _Web API_:
<div align="center">
    <p><b>docker run --rm --init --ulimit core=0 -p 8070:8070 grobid/grobid:0.8.1</b></p>
</div>

Infine, attraverso un qualsiasi _browser_, è possibile accertarsi se il servizio operi correttamente, accedendo alla pagina dedicata alla _console_ di _Grobid_, esposta tramite l'_URL https://localhost:8070_. La risposta ricevuta dal servizio consiste in un insieme di file _XML_, i quali sono memorizzati all'interno della cartella _TEI_.

In [20]:
from grobid_client.grobid_client import GrobidClient

grobid_client = GrobidClient(config_path="../json/grobid/config.json")
grobid_client.process("processFulltextDocument", input_dir, output=output_dir, n=20)

GROBID server is up and running


In [21]:
def define_pdf_paths() -> List[str]:
    path_files = []

    for path in os.listdir(output_dir):
        path_files.append(path)

    return path_files

xml_paths = define_pdf_paths()

In [29]:
import re

from bs4 import BeautifulSoup

# Extracting content from XML, after defining a list of progressive tags
def extract_content(soup: BeautifulSoup, tags: List[str]) -> str | None:
    try:
        element = soup.find(tags[0])

        for tag in tags[1:]:
            element = element.find(tag)

        return element.contents[0]
    except Exception:
        return None
    
# Function used to detect authors' forename and username by RegEx
def extract_name(expression: str, persName) -> str | None:
    try:
        match = re.search(expression, str(persName))

        return match.group(1)
    except Exception:
        return None

# BeatifulSoup consists in a data structure representing a parsed XML file
def extract_field_from_xml(path_files: List[str]) -> List[Metadata]:
    list_authors: List[Author] = []
    list_metadata: List[Metadata] = []
    
    for path in sorted(path_files):
        try: 
            if path.endswith(".xml"):
                with open(output_dir + path, "r") as content:
                    xml = content.read()

                soup = BeautifulSoup(xml, "xml")
                
                title = extract_content(soup, ["titleStmt", "title"])
                abstract = extract_content(soup, ["profileDesc", "abstract", "p"])
        
                try:
                    introduction = extract_content(soup, ["body", "div", "p"])
                except Exception:
                    introduction = None

                try: 
                    persNames = soup.find("sourceDesc").find("biblStruct").find("analytic").find_all("persName")
                except Exception:
                    persNames = None

                if persNames is not None:
                    for persName in persNames:
                        forename = extract_name(r"<forename\stype=\"first\">(.*?)<\/forename>", persName)
                        surname = extract_name(r"<surname>(.*?)<\/surname>", persName)
                        
                        if None not in (forename, surname):
                            author = Author(forename, surname)
                            list_authors.append(author)

                list_metadata.append(Metadata(DOI=None, path=path, title=title, author=list_authors, abstract=abstract, introduction=introduction))
                list_authors = []
                
            continue
        except Exception as e:
            print("Error during parsing ", path," file: ", e)

    return list_metadata

list_metadata = extract_field_from_xml(xml_paths)

Nonostante l'applicazione di _Grobid_, alcuni PDF sono privi di _metadati_. Pertanto, affinchè l'analisi condotta possa essere estesa ad un numero sempre più vasto di file,
è stata implementata la libreria _pypdf_; quest'ultima, permette di manipolare ogni singola pagina che componga il PDF. A tal proposito, sono acquisite ulteriori informazioni disponibili tramite il _field metadata_.

In [23]:
from pypdf import PdfReader

def define_authors(str_author: str) -> List[Author]:
    list_authors: List[Metadata] = []
    try:
        authors = re.split(r"\sand\s", str_author)

        for author in authors:
            items = author.split(" ")
            list_authors.append(Author(items[0], items[1]))

        return list_authors
    except Exception:
        return list_authors

def retrieve_missing_metadata(list_metadata: List[Metadata]):
    for metadata in list_metadata:
        if metadata.title is None or len(metadata.author) == 0:
            pdf_path = re.search(r"(^.*)?.grobid", metadata.path).group(1) + ".pdf"

            _pdfreader = PdfReader(input_dir + pdf_path).metadata
            
            title = _pdfreader.title or None
            author = _pdfreader.author or []

            metadata.title = title
            metadata.path = pdf_path
            metadata.author = define_authors(author)

retrieve_missing_metadata(list_metadata)

parsing for Object Streams
parsing for Object Streams
parsing for Object Streams
parsing for Object Streams
parsing for Object Streams


In [ ]:
import time
import requests

from crossref.restful import Works
from difflib import SequenceMatcher

def get_authors_string(list_authors: List[Author]) -> List[str]:
    str_authors = ""
    for i in range(0, len(list_authors)):
        if i >= len(list_authors):
            str_authors += list_authors[i].forename + " " + list_authors[i].surname
        else:
            str_authors += list_authors[i].forename + " " + list_authors[i].surname + " "

    return str_authors

def similar(str_1: str, str_2: str) -> float:
    return SequenceMatcher(None, str_1, str_2).ratio()

work = Works()

def send_crossref_request(title: str, list_authors: List[str]) -> str:
    url_request = work.query(bibliographic=title, author=get_authors_string(list_authors)).url

    time.sleep(2)

    try:
        response = requests.get(url_request)

        match response.status_code:
            case 200:
                content = response.json()
                message = content.get("message")

                items = message.get("items")
                for item in items:
                    if similar(title, item["title"][0]) > 0.5:
                        return item["DOI"]
                                                                
                    continue
            case 400:
                raise Exception("Error during request to CrossRef REST API: Bad Request")
            case _:
                raise Exception("Error during request to CrossRef REST API: WTF")
    except Exception as e:
        print("Error during request to CrossRef REST API:", e)

for metadata in list_metadata:
    if None in (metadata.title, metadata.author):
        print(metadata.path)
    else:
        doi = send_crossref_request(metadata.title, metadata.author)

        if doi is not None:
            metadata.DOI = doi

In [25]:
list_not_found_dois = list(filter(lambda metadata: metadata.DOI is None, list_metadata))
list_found_dois = list(filter(lambda metadata: metadata.DOI is not None, list_metadata))

La sezione finale del notebook si concentra su alcuni controlli attuati per accertarsi della __bontà__ dei _Digital Object Identifier_ ricavati. Ciò avviene interrogando nuovamente la _REST API_ di _CrossRef_. Tuttavia, in questa casistica, non sono combinati il _title_ e gli _authors_ estrapolati precedentemente, attraverso apposite librerie, ma è inserito all'interno dell'_URL_ lo stesso _DOI_; in questo modo, è possibile definire il grado di similarità tra le informazioni già in possesso rispetto ai nuovi dati ottenuti dalla risposta dell'_Application Programming Interface_.

Di seguito, sono riportate le operazioni principali del comportamento descritto, suddivise in:
- __Definizione della lista__, lista contenente l'insieme di tutti i _metadati_ riferiti ai PDF contenuti nella _directory articlesGrobid_
- __Invio della richiesta__, richiesta riferita alla _REST API_ di _CrossRef_, da cui, qualora sia positiva la risposta, saranno ricavati i "nuovi" dati necessari per il confronto
- __Definizione grado di similarità__, tramite la libreria _SequenceMatcher_ è estrapolato il _grado di similarità_ del _metadato_ analizzato, qualora dovesse essere maggiore di _0.6_, dato che si tratta di un valore decimale appartenente all'intervallo _[0, 1]_, è rimosso dalla lista

In [26]:
# Small check to detect the correctness of metadata
def define_wrong_dois(list_metadata: List[Metadata]) -> List[Metadata]:
    # List that will contain the "wrong" DOIs retrieved previously
    list_wrong_dois = list(filter(lambda metadata: metadata.DOI is not None, list_metadata))

    for metadata in list_metadata:
        if metadata.DOI is not None:
            url_request = work.query().url + "/" + metadata.DOI
        
            try:
                response = requests.get(url_request)

                match response.status_code:
                    case 200:
                        content = response.json()

                        try:
                            message = content.get("message")

                            # Another check may concern the authors and the publication date
                            titles = message.get("title")
                            authors = message.get("author")
                
                            for title in titles:
                                if similar(metadata.title, title) > 0.6:
                                    list_wrong_dois.remove(metadata)
                                    break

                                continue

                        except Exception as e:
                            print("Field not found", metadata.path)
                            continue
                    case 400:
                        raise Exception("Error during request to CrossRef REST API: Bad Request")
                    case _:
                        raise Exception("Error during request to CrossRef REST API: WTF")
            except Exception as e:
                print("Error during request to CrossRef REST API:", e)

    return list_wrong_dois

list_wrong_dois = define_wrong_dois(list_metadata)
list_correct_dois = list(filter(lambda metadata: metadata in list_found_dois and metadata not in list_wrong_dois, list_metadata))

In [30]:
import json

def define_list(_list: List[Metadata]) -> List:
    list_json = []

    for item in _list:
        list_json.append(item.get_dict())

    return list_json

with open("../json/grobid/summary.json", "w") as file:
    json.dump(define_list(list_metadata), file, indent=3)

In [28]:
import pandas 

dict_doi_summary = {
    "Total": len(list_metadata),
    "Found": len(list_found_dois),
    "Not found": len(list_not_found_dois)
}

dict_doi_accuracy = {
    "Found": len(list_found_dois),
    "Correct": len(list_correct_dois),
    "Wrong": len(list_wrong_dois)
}

df_doi_summary = pandas.DataFrame.from_dict(dict_doi_summary, orient="index", columns=["# DOI"])
display(df_doi_summary)

df_doi_accuracy = pandas.DataFrame.from_dict(dict_doi_accuracy, orient="index", columns=["# DOI"])
display(df_doi_accuracy)

,# DOI
Total,76
Found,46
Not found,30


,# DOI
Found,46
Correct,37
Wrong,9
